## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-08-21-49-39 +0000,missionlocal,New Legion of Honor show celebrates Ferlinghet...,https://missionlocal.org/2025/09/new-legion-of...
1,2025-09-08-21-48-45 +0000,nyt,Lachlan Murdoch Secures Control of Fox and New...,https://www.nytimes.com/2025/09/08/business/me...
2,2025-09-08-21-45-22 +0000,wapo,Hegseth in Puerto Rico as Pentagon eyes island...,https://www.washingtonpost.com/national-securi...
3,2025-09-08-21-45-08 +0000,bbc,Murdochs reach deal in succession battle over ...,https://www.bbc.com/news/articles/cn825x71g4do...
4,2025-09-08-21-45-03 +0000,nypost,Mother and 2-month-old baby killed in massive ...,https://nypost.com/2025/09/08/world-news/mothe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2318/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,34
0,new,17
44,court,13
48,immigration,11
88,raid,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
130,2025-09-08-17-20-00 +0000,wsj,New polling by the Federal Reserve Bank of New...,https://www.wsj.com/economy/jobs/employment-su...,92
166,2025-09-08-14-55-27 +0000,nypost,Trump demands foreign firms in US ‘hire and tr...,https://nypost.com/2025/09/08/business/trump-d...,80
205,2025-09-08-11-26-15 +0000,cbc,At least 19 killed as police open fire on Nepa...,https://www.cbc.ca/news/world/nepal-deadly-pro...,78
183,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...,76
124,2025-09-08-17-37-04 +0000,nypost,Supreme Court lets Trump fire Democratic FTC C...,https://nypost.com/2025/09/08/us-news/supreme-...,73


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
130,92,2025-09-08-17-20-00 +0000,wsj,New polling by the Federal Reserve Bank of New...,https://www.wsj.com/economy/jobs/employment-su...
166,80,2025-09-08-14-55-27 +0000,nypost,Trump demands foreign firms in US ‘hire and tr...,https://nypost.com/2025/09/08/business/trump-d...
205,78,2025-09-08-11-26-15 +0000,cbc,At least 19 killed as police open fire on Nepa...,https://www.cbc.ca/news/world/nepal-deadly-pro...
48,53,2025-09-08-20-43-00 +0000,wsj,Supreme Court Lifts Limits on Immigration Raid...,https://www.wsj.com/us-news/law/supreme-court-...
183,42,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...
285,40,2025-09-08-01-00-02 +0000,cbc,Australian woman gets life in prison for fatal...,https://www.cbc.ca/news/world/erin-patterson-l...
110,40,2025-09-08-18-09-33 +0000,nypost,Mexico City train crash leaves at least 10 dea...,https://nypost.com/2025/09/08/world-news/train...
305,39,2025-09-07-22-34-59 +0000,latimes,"South Korea, irked at U.S. raid at Hyundai pla...",https://www.latimes.com/business/story/2025-09...
101,37,2025-09-08-18-28-37 +0000,nypost,CBS News taps former CEO of conservative thin...,https://nypost.com/2025/09/08/media/cbs-news-t...
118,33,2025-09-08-17-59-49 +0000,nypost,Beloved Bronx ‘hole-in-the-wall’ pizzeria name...,https://nypost.com/2025/09/08/lifestyle/bronx-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
